In [2]:
import os
import pandas as pd
import base64
from dash import Dash, dcc, html, dash_table, callback_context
import dash_leaflet as dl
from dash.dependencies import Input, Output, State
import plotly.express as px
from pymongo import MongoClient
from jupyter_dash import JupyterDash

def load_data():
    USER = 'aacuser'
    PASS = 'SNHU1234'
    HOST = 'nv-desktop-services.apporto.com'
    PORT = 31580
    DB = 'aac'
    COL = 'animals'

    try:
        # Initialize Connection
        client = MongoClient(f'mongodb://{USER}:{PASS}@{HOST}:{PORT}')
        database = client[DB]
        collection = database[COL]

        # Fetch data from MongoDB
        data = list(collection.find())
        
        if not data:
            raise ValueError("No data found in the MongoDB collection")
        
        df = pd.DataFrame(data)
        
        print(f"Successfully loaded data from MongoDB")
        print(f"DataFrame shape: {df.shape}")
        print("Columns:", df.columns.tolist())
        print("\nFirst few rows of the data:")
        print(df.head())
        
        # Check for required columns
        required_columns = ['animal_type', 'breed', 'color', 'age_upon_outcome', 'sex_upon_outcome', 'age_upon_outcome_in_weeks']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"Warning: The following required columns are missing: {missing_columns}")
        
    except Exception as e:
        print(f"An error occurred while connecting to MongoDB: {e}")
        df = pd.DataFrame()
    
    return df

# Load the data
df = load_data()

# Initialize the JupyterDash app
app = JupyterDash(__name__)

# Load and encode the logo
try:
    image_filename = 'logo.png'
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
except Exception as e:
    print(f"Error loading logo: {e}")
    encoded_image = None

# Define the layout
app.layout = html.Div([
    html.Div([
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '100px', 'display': 'inline-block', 'vertical-align': 'middle'}) if encoded_image else html.Div(),
        html.H1('SNHU CS-340 Grazioso Salvare Dashboard', style={'display': 'inline-block', 'vertical-align': 'middle', 'margin-left': '20px'})
    ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center', 'margin': '20px'}),
    html.Hr(),
    html.Div([
        html.Button('Water Rescue', id='water-rescue-button', n_clicks=0),
        html.Button('Mountain/Wilderness Rescue', id='mountain-rescue-button', n_clicks=0),
        html.Button('Disaster/Individual Tracking', id='disaster-rescue-button', n_clicks=0),
        html.Button('Reset', id='reset-button', n_clicks=0),
    ], style={'textAlign': 'center', 'margin': '20px'}),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns if i != '_id'],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div([
        html.Div([
            dcc.Graph(id='pie-chart')
        ], style={'width': '50%', 'display': 'inline-block'}),
        html.Div([
            html.Div(id='map-id')
        ], style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'top'})
    ])
])

@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'selected_rows')],
    [Input('water-rescue-button', 'n_clicks'),
     Input('mountain-rescue-button', 'n_clicks'),
     Input('disaster-rescue-button', 'n_clicks'),
     Input('reset-button', 'n_clicks')]
)
def update_dashboard(water_clicks, mountain_clicks, disaster_clicks, reset_clicks):
    ctx = callback_context
    if not ctx.triggered:
        return df.to_dict('records'), [0]
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if button_id == 'water-rescue-button':
        filtered_df = df[
            (df['animal_type'] == 'Dog') &
            (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
            (df['sex_upon_outcome'] == 'Intact Female') &
            (df['age_upon_outcome_in_weeks'].between(26, 156))
        ]
    elif button_id == 'mountain-rescue-button':
        filtered_df = df[
            (df['animal_type'] == 'Dog') &
            (df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'].between(26, 156))
        ]
    elif button_id == 'disaster-rescue-button':
        filtered_df = df[
            (df['animal_type'] == 'Dog') &
            (df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'].between(20, 300))
        ]
    else:  # reset
        filtered_df = df

    return filtered_df.to_dict('records'), [0]

@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_graph(data):
    dff = pd.DataFrame(data)
    fig = px.pie(dff, names='breed', title='Distribution of Dog Breeds')
    return fig

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, selected_rows):
    if not viewData:
        return html.Div("No data available")
    
    dff = pd.DataFrame(viewData)
    row = 0 if selected_rows is None or len(selected_rows) == 0 else selected_rows[0]
    
    try:
        lat = float(dff.iloc[row]['location_lat'])
        lon = float(dff.iloc[row]['location_long'])
        return [
            dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(dff.iloc[row]['animal_id']),
                    dl.Popup([
                        html.H1("Animal Details"),
                        html.P(f"Breed: {dff.iloc[row]['breed']}"),
                        html.P(f"Age: {dff.iloc[row]['age_upon_outcome']}"),
                        html.P(f"Sex: {dff.iloc[row]['sex_upon_outcome']}")
                    ])
                ])
            ])
        ]
    except Exception as e:
        print(f"Error updating map: {e}")
        return html.Div("Unable to display map due to missing or invalid data")

# Run the app
if __name__ == '__main__':
    app.run_server(mode='external', host='0.0.0.0', port=8050, debug=True)

An error occurred while connecting to MongoDB: nv-desktop-services.apporto.com:31580: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 66c2708c5807c6d4fa0dc5d2, topology_type: Single, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 31580) server_type: Unknown, rtt: None, error=AutoReconnect('nv-desktop-services.apporto.com:31580: [Errno 111] Connection refused')>]>


ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=8050): Max retries exceeded with url: /_alive_54337f7f-dce7-4228-8b6b-b2d95a909432 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f3676c50ac0>: Failed to establish a new connection: [Errno 111] Connection refused'))